In [1]:
import sys
import os
import json
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style()
import pandas as pd

# import dysts
sys.path.append(os.path.join(os.getcwd(),'../../darts/'))
import darts

sys.path.append(os.path.join(os.getcwd(),'../../dysts/'))
from dysts.datasets import *
from dysts.flows import *
from dysts.base import *
from dysts.utils import *
from dysts.analysis import *

#import degas as dg
#dg.set_style()

%matplotlib inline
%load_ext autoreload
%autoreload 2

NvvmError: Failed to compile

IR version 1.6 incompatible with current version 2.0
<unnamed>: error: incompatible IR detected. Possible mix of compiler/IR from different releases.
NVVM_ERROR_IR_VERSION_MISMATCH

# Tune hyperparameters

In [ ]:
%%bash

# python3 find_hyperparameters.py

# Run and record forecasts on test data

In [ ]:
%%bash --out output --err error

# python3 compute_benchmarks.py

## Load Forecasting Benchmark Results

In [ ]:
import os
import json

GRANULARITY = 100
DATE = 221025 # noise level 0.8, not including RBF
#DATE = 220812 # noise level 0.2, not including RBF

#/results/221025_results_test_univariate__pts_per_period_ # large noise 
#/results/220812_results_test_univariate__pts_per_period_ # small noise 

#220315_results_test_univariate__pts_per_period_ #Initial try
#220326_results_test_univariate__pts_per_period_
#/results/220426_results_test_univariate__pts_per_period_

with open(
    os.getcwd()
    + "/results/" + str(DATE) + "_results_test_univariate__pts_per_period_" + str(GRANULARITY) 
    + "__periods_12_noise.json",
    "r",
) as file:
    all_results = json.load(file)

#220428
# with open(os.getcwd() + "/results/results_test_univariate__pts_per_period_15__periods_12_noise.json", "r") as file:
#     all_results = json.load(file)



## Convert R2 into a distance
all_r2 = list()
for equation_name in all_results:
    for model_name in all_results[equation_name]:
        if model_name == "values":
            continue            
        else:
            ## convert r2 to a pseudodistance
            all_results[equation_name][model_name]["r2_score"] = (
                1 - all_results[equation_name][model_name]["r2_score"]
            )

            ## Coefficient of variation must be normed
            all_results[equation_name][model_name]["coefficient_of_variation"] = np.abs(
                all_results[equation_name][model_name]["coefficient_of_variation"]
            )

            ## Drop RMSE because it overlaps with MSE
            all_results[equation_name][model_name].pop("rmse", None)

# HS Remove unimportant methods
for equation_name in all_results:
    for model_name in ['FFT', 'FourTheta', 'AutoARIMA', 'LSS_Takens']:
        #all_results[equation_name].pop(model_name, None)
        del all_results[equation_name][model_name]

# HS using local darts
sys.path.append(os.path.join(os.getcwd(),'../../darts/'))
import darts
from darts import TimeSeries
from darts.metrics import mae

wrap_array = lambda x: TimeSeries.from_dataframe(pd.DataFrame(np.squeeze(np.array(x))))
mae_func = lambda x, y: mae(wrap_array(x), wrap_array(y))


## Calculate MASE
for equation_name in all_results:
    baseline_onestep = mae_func(
        all_results[equation_name]["values"][1:],
        all_results[equation_name]["values"][:-1],
    )
    for model_name in all_results[equation_name]:
        if model_name == "values":
            continue
        else:
            mae_val = all_results[equation_name][model_name]["mae"]
            all_results[equation_name][model_name]["mase"] = mae_val / baseline_onestep

## get best models
all_best_models = list()
for equation_name in all_results:
    all_models = list()
    all_smapes = list()
    for model_name in all_results[equation_name]:
        if model_name != "values":
            all_models.append(model_name)
            all_smapes.append(all_results[equation_name][model_name]["smape"])
    all_best_models.append(all_models[np.argmin(all_smapes)])

all_results_transposed = dict()
for model_name in all_results["Lorenz"]:
    if model_name == "values":
        continue
    all_results_transposed[model_name] = dict()
for equation_name in all_results:
    for model_name in all_results[equation_name]:
        if model_name == "values":
            continue
        all_results_transposed[model_name][equation_name] = all_results[equation_name][
            model_name
        ]


hist_values = dict()
for model_name in all_results_transposed:
    smape_vals = list()
    for key in all_results_transposed[model_name]:
        smape_vals.append(all_results_transposed[model_name][key]["smape"])

    hist_values[model_name] = smape_vals.copy()

if GRANULARITY == 15:
    hist_values_low = hist_values.copy()
elif GRANULARITY == 100:
    hist_values_high = hist_values.copy()
else:
    raise ValueError("Granularity must be 15 or 100 points per period.")
    

    #/results/221025_results_test_univariate__pts_per_period_
#/results/220812_results_test_univariate__pts_per_period_ # small noise 

if DATE == 220812:
    hist_values_low = hist_values.copy()
elif DATE == 221025:
    hist_values_high = hist_values.copy()
else:
    raise ValueError("Granularity must be 15 or 100 points per period.")


all_metric_names = list(all_results_transposed["ARIMA"]["Aizawa"].keys())
all_metric_names.remove("prediction")

## Compute forecast metrics
metric_records = dict()
for metric_name in all_metric_names:
    metric_records[metric_name] = list()
for model_name in all_results_transposed:
    for equation_name in all_results_transposed[model_name]:
        for metric_name in all_metric_names:
            metric_records[metric_name].append(
                all_results_transposed[model_name][equation_name][metric_name]
            )
df_metrics = pd.DataFrame(metric_records)


# df_metrics["coefficient_of_variation"] = np.abs(df_metrics["coefficient_of_variation"])
# df_metrics = df_metrics.drop('rmse', 1)

### Load mathematical properties

In [ ]:
import dysts.flows

max_lyap = list()
best_scores = list()
for equation_name in all_results:    
    all_scores_per_equation = list()
    for model_name in all_results[equation_name]:
        if model_name == "values":
            continue
        all_scores_per_equation.append(all_results[equation_name][model_name]["smape"])
    
    best_scores.append(min(all_scores_per_equation))

    eq = getattr(dysts.flows, equation_name)()
    max_lyap.append(eq.maximum_lyapunov_estimated)
        


### Correlate forecasting models

In [ ]:
model_records = dict()
for model_name in all_results_transposed.keys():
    model_records[model_name] = list()
    
for model_name in all_results_transposed:
    for equation_name in all_results_transposed[model_name]:
        model_records[model_name].append(all_results_transposed[model_name][equation_name]["smape"])    
    
df_models = pd.DataFrame(model_records)


corr_array = np.array(df_models.corr(method='spearman'))
np.fill_diagonal(corr_array, np.nan)

df_models_sorted = df_models.iloc[:, np.argsort(np.nanmedian(corr_array, axis=0))[::-1]]
df_models_sorted = df_models.iloc[:, np.argsort(np.nanmax(corr_array, axis=0))[::-1]]

# df_models_sorted = df_models.iloc[:, sort_order]

ax = sns.heatmap(df_models_sorted.corr(method='spearman'), cmap="mako", vmin=0, vmax=1)
ax.set_aspect('equal')
ax.tick_params(axis='both', which='both', length=0)

# dg.better_savefig("../private_writing/fig_resources/model_correlation_map.png", dpi=600)

### Compare forecasting metrics

In [ ]:

corr_array = np.array(df_metrics.corr(method='spearman'))
np.fill_diagonal(corr_array, np.nan)

metric_sort_inds = np.argsort(np.nanmedian(corr_array, axis=0))[::-1]
metric_sort_inds = np.argsort(np.nanmax(corr_array, axis=0))[::-1]
df_metrics_sorted = df_metrics.iloc[:, metric_sort_inds]


ax = sns.heatmap(df_metrics_sorted.corr(method='spearman'), cmap="mako", vmin=0, vmax=1)
ax.set_aspect('equal')
ax.tick_params(axis='both', which='both', length=0)

# dg.better_savefig("../private_writing/fig_resources/metric_map.png", dpi=600)

### Compare forecasting results against mathematical properties

In [ ]:
chosen_metric = "smape"
models_df = dict()
metrics_df = dict()
for equation_name in all_results:
    models_df[equation_name] = dict()
    metrics_df[equation_name] = dict()
    for model_name in all_results[equation_name]:
        if model_name == "values": continue
        models_df[equation_name][model_name] = all_results[equation_name][model_name][chosen_metric]
    
        for metric_name in all_results[equation_name]["ARIMA"]:
            if metric_name == "prediction": continue
            metrics_df[equation_name][metric_name] = all_results[equation_name]["NBEATSModel"][metric_name]
    
    
models_df = pd.DataFrame(models_df).transpose()
metrics_df = pd.DataFrame(metrics_df).transpose()

In [ ]:
import dysts.flows

attributes =  ['maximum_lyapunov_estimated', 'kaplan_yorke_dimension', 'multiscale_entropy', 'correlation_dimension']
all_properties = dict()
for equation_name in get_attractor_list():
    if equation_name in ['GenesioTesi','Hadley','SprottD','StickSlipOscillator']:
        continue
    
    eq = getattr(dysts.flows, equation_name)()
    attr_vals = [getattr(eq, item, None) for item in attributes]
    
    all_properties[equation_name] = dict(zip(attributes, attr_vals))
    
all_properties = pd.DataFrame(all_properties).transpose().dropna()

In [ ]:
cross_corr = pd.concat([metrics_df, all_properties], axis=1, keys=['metrics_df', 'all_properties']).corr(method="spearman").loc['metrics_df', 'all_properties']
math_sort_inds = np.argsort(np.max(cross_corr, axis=0))[::-1]

ax = sns.heatmap(cross_corr.transpose().iloc[math_sort_inds, metric_sort_inds], cmap="mako", vmin=0, vmax=1)
ax.set_aspect(1)
ax.tick_params(axis='both', which='both', length=0)

# dg.better_savefig("../private_writing/fig_resources/metric_math.png", dpi=600)

In [ ]:
cross_corr = pd.concat([models_df, all_properties], axis=1, keys=['models_df', 'all_properties']).corr(method="spearman").loc['models_df', 'all_properties']
math_sort_inds = np.argsort(np.max(cross_corr, axis=0))[::-1]

ax = sns.heatmap(cross_corr.transpose().iloc[math_sort_inds][df_models_sorted.columns], cmap="mako", vmin=0, vmax=1)
ax.set_aspect(1)
ax.tick_params(axis='both', which='both', length=0)

# dg.better_savefig("../private_writing/fig_resources/models_math_corr.png", dpi=600)

In [ ]:

## Easiest and hardest systems
print("Easiest systems: ", np.unique(models_df.idxmin(axis=0), return_counts=True))
print("Hardest systems: ", np.unique(models_df.idxmax(axis=0), return_counts=True))


all_median_names = list()
for key in models_df.keys():
    all_median_names.append(models_df[models_df[key] == models_df[key].quantile(interpolation='nearest')].index[0])
    
print("Middle systems: ", np.unique(all_median_names, return_counts=True))

### Compare different forecasting models

In [ ]:
import pandas as pd

def mirror_df(df, mirror_val=0):
    """
    Create a mirrored augmented dataframe. Used
    for setting the right boundary conditions on kernel 
    density plots
    """
    if np.isscalar(mirror_val):
        return pd.concat([df, mirror_val - df])
    else:
        all_out_df = [df]
        for val in mirror_val:
            all_out_df.append(val - df)
        return pd.concat(all_out_df)
    
#dflo = pd.DataFrame.from_dict(hist_values_low) ##########Commented out! HS
dflo = pd.DataFrame.from_dict(hist_values_high)
dflo["Granularity"] = "Coarse"

dfhi = pd.DataFrame.from_dict(hist_values_high)
dfhi["Granularity"] = "Fine"
#data = pd.merge(dflo, dfhi, how="outer")
data = pd.merge(dfhi, dflo, how="outer")

all_model_names = np.array(list(hist_values_high.keys()))
all_medians = [np.median(hist_values_high[name]) for name in hist_values_high]
all_means = [np.mean(hist_values_high[name]) for name in hist_values_high]
#sort_order = np.argsort(all_medians)
sort_order = np.argsort(all_means)

data_long = pd.melt(data, value_vars=all_model_names[sort_order], id_vars="Granularity")

In [ ]:
data_long

In [ ]:
plt.figure(figsize=(20, 10))
ax = plt.gca()


ax = sns.violinplot(
    data=data_long,
    x="variable",
    y="value",
    hue="Granularity",
    order=all_model_names[sort_order],
    linewidth=0,
    size=3,
    alpha=0.1,
    split=True,
    scale="area",
    inner=None,
    palette="mako",
)
ax = sns.pointplot(
    data=data_long,
    x="variable",
    y="value",
    hue="Granularity",
    order=all_model_names[sort_order],
    linewidth=0,
    size=4,
    palette={"Coarse": "w", "Fine": "w"},
    join=False,
    ci=99,
    dodge=0.2,
)


plt.ylim([0, 200])
#dg.fixed_aspect_ratio(1 / 8)
ax.grid(False)
ax.legend_.remove()

# dg.better_savefig("../private_writing/fig_resources/forecasting_violins.png", dpi=600)

In [ ]:
#dflo = pd.DataFrame.from_dict(hist_values_low) ##########Commented out! HS
dflo = pd.DataFrame.from_dict(hist_values_high)
dflo["Granularity"] = "Coarse"

dfhi = pd.DataFrame.from_dict(hist_values_high)
dfhi["Granularity"] = "Fine"
#data = pd.merge(dflo, dfhi, how="outer")
data = pd.merge(dfhi, dflo, how="outer")

all_model_names = np.array(list(hist_values_high.keys()))
all_medians = [np.median(hist_values_high[name]) for name in hist_values_high]
all_means = [np.mean(hist_values_high[name]) for name in hist_values_high]
#sort_order = np.argsort(all_medians)
sort_order = np.argsort(all_means)

#data_long = pd.melt(data, value_vars=all_model_names[sort_order], id_vars="Granularity")
data_long = pd.melt(dfhi, value_vars=all_model_names[sort_order], id_vars="Granularity")

plt.figure(figsize=(20, 10))
ax = plt.gca()


ax = sns.violinplot(
    data=data_long,
    x="variable",
    y="value",
    order=all_model_names[sort_order],
    #hue = ,
    linewidth=0,
    size=3,
    alpha=0.1,
    split=False,
    scale="area",
    inner=None,
    palette="flare_r",
)

ax = sns.pointplot(
    data=data_long,
    x="variable",
    y="value",
    order=all_model_names[sort_order],
    linewidth=0,
    size=4,
    color = "w",
    join=False,
    ci=99,
    dodge=0.2,
)


plt.ylim([0, 200])
#dg.fixed_aspect_ratio(1 / 8)
ax.grid(False)

In [ ]:
print(np.argsort(all_means))

In [ ]:
all_means

In [ ]:
all_medians

In [ ]:
#median

#dflo = pd.DataFrame.from_dict(hist_values_low) ##########Commented out! HS
dflo = pd.DataFrame.from_dict(hist_values_high)
dflo["Granularity"] = "Coarse"

dfhi = pd.DataFrame.from_dict(hist_values_high)
dfhi["Granularity"] = "Fine"
#data = pd.merge(dflo, dfhi, how="outer")
data = pd.merge(dfhi, dflo, how="outer")

all_model_names = np.array(list(hist_values_high.keys()))
all_medians = [np.median(hist_values_high[name]) for name in hist_values_high]
all_means = [np.mean(hist_values_high[name]) for name in hist_values_high]
#sort_order = np.argsort(all_medians)
sort_order = np.argsort(all_means)

#data_long = pd.melt(data, value_vars=all_model_names[sort_order], id_vars="Granularity")
data_long = pd.melt(dfhi, value_vars=all_model_names[sort_order], id_vars="Granularity")

plt.figure(figsize=(20, 4))
ax = plt.gca()


ax = sns.violinplot(
    data=data_long,
    x="variable",
    y="value",
    order=all_model_names[sort_order],
    #linewidth=0,
    size=3,
    alpha=0.1,
    split=False,
    scale="area",
    inner="quart",
    #palette="flare_r",
)

ax = sns.pointplot(
    data=data_long,
    x="variable",
    y="value",
    order=all_model_names[sort_order],
    linewidth=0,
    size=4,
    color = "w",
    join=False,
    ci=0,
    dodge=0.2,
    estimator = np.mean,
)


plt.ylim([0, 250])
#dg.fixed_aspect_ratio(1 / 8)
ax.grid(False)

In [ ]:

plt.figure(figsize=(20, 4))
ax = plt.gca()


ax = sns.violinplot(
    data=data_long,
    x="variable",
    y="value",
    order=all_model_names[sort_order],
    size=3,
    alpha=0.1,
    split=False,
    scale="area",
    # inner=None,
    inner="points",
    # palette="flare_r",
)

plt.ylim([0, 200])
#dg.fixed_aspect_ratio(1 / 8)
#ax.grid(False)

In [ ]:
# #dflo = pd.DataFrame.from_dict(hist_values_low) ##########Commented out! HS
# dflo = pd.DataFrame.from_dict(hist_values_high)
# dflo["Granularity"] = "Coarse"

# dfhi = pd.DataFrame.from_dict(hist_values_high)
# dfhi["Granularity"] = "Fine"
# #data = pd.merge(dflo, dfhi, how="outer")
# data = pd.merge(dfhi, dflo, how="outer")

# all_model_names = np.array(list(hist_values_low.keys()))
# all_medians = [np.median(hist_values_low[name]) for name in hist_values_low]
# all_means = [np.mean(hist_values_low[name]) for name in hist_values_low]
# #sort_order = np.argsort(all_medians)
# sort_order = np.argsort(all_means)

# #data_long = pd.melt(data, value_vars=all_model_names[sort_order], id_vars="Granularity")
# data_long = pd.melt(dflo, value_vars=all_model_names[sort_order], id_vars="Granularity")

# plt.figure(figsize=(20, 10))
# ax = plt.gca()


# ax = sns.violinplot(
#     data=data_long,
#     x="variable",
#     y="value",
#     order=all_model_names[sort_order],
#     linewidth=0,
#     size=3,
#     alpha=0.1,
#     split=False,
#     scale="area",
#     inner=None,
#     palette="flare_r",
# )

# ax = sns.pointplot(
#     data=data_long,
#     x="variable",
#     y="value",
#     order=all_model_names[sort_order],
#     linewidth=0,
#     size=4,
#     color = "w",
#     join=False,
#     ci=99,
#     dodge=0.2,
# )


# plt.ylim([0, 200])
# #dg.fixed_aspect_ratio(1 / 8)
# ax.grid(False)

# #https://seaborn.pydata.org/tutorial/color_palettes.html

In [ ]:
all_model_names

In [ ]:
data_long

## Group models by mathematical properties

In [ ]:
equation_names = list(all_results.keys())
model_names = list(all_results[equation_names[0]].keys())
model_names.remove("values")
score_names = list(all_results[equation_names[0]][model_names[0]].keys())
score_names.remove("prediction")

results_reduced = dict()
for equation_name in equation_names:
    if equation_name in ['GenesioTesi','Hadley','SprottD','StickSlipOscillator']:
        continue
    results_reduced[equation_name] = dict()
    for model_name in model_names:
        val = all_results[equation_name][model_name]["smape"]
        results_reduced[equation_name][model_name] = val 
results_reduced = pd.DataFrame.from_dict(results_reduced)     
results_reduced = results_reduced.iloc[sort_order] # sort by median error

In [ ]:

sort_inds = all_properties.sort_values(by="maximum_lyapunov_estimated").index

color_list = sns.color_palette("husl", results_reduced.shape[0])

plt.figure()
for clr, row in zip(color_list, results_reduced[sort_inds].to_numpy()):
    plt.semilogx(
        all_properties["maximum_lyapunov_estimated"][sort_inds],
        row,
        color=clr
    )
plt.title("Score versus Lyapunov exponent")



sort_inds = all_properties.sort_values(by="correlation_dimension").index

plt.figure()
for clr, row in zip(color_list, results_reduced[sort_inds].to_numpy()):
    plt.plot(
        all_properties["correlation_dimension"][sort_inds],
        row,
        color=clr
    )
plt.title("Score versus correlation dimension")


plt.figure()
for clr, row in zip(color_list, results_reduced[sort_inds].to_numpy()):
    plt.plot(
        all_properties["correlation_dimension"][sort_inds],
        row,
        color=clr,
        linewidth=4
    )
plt.legend(results_reduced.index, facecolor='white', framealpha=1, frameon=True)
plt.title("Show legend")

In [ ]:
#prop_name = "correlation_dimension"
prop_name = "maximum_lyapunov_estimated"
#prop_name = "multiscale_entropy"

all_vals = list()
vals = np.array(all_properties[prop_name])

all_windows = list(zip(10 * np.arange(9), 10 * np.arange(2, 11)))

for window in all_windows:
    sel_inds = all_properties[prop_name][
        all_properties[prop_name].between(
            np.percentile(vals, window[0]), np.percentile(vals, window[1])
        )
    ].index
    all_vals.append(results_reduced[sel_inds].median(axis=1))
    
plt.figure()
for clr, row in zip(color_list, np.array(all_vals).T):
    plt.plot(row, color=clr)
#plt.ylim([0, 180])

plt.legend(results_reduced.index, facecolor='white', framealpha=1, frameon=True)

#dg.better_savefig("../private_writing/fig_resources/" + prop_name + "_ranks.png", dpi=600)

In [ ]:

    
plt.figure()
for clr, row in zip(color_list, results_reduced[sort_inds].to_numpy()):
    plt.plot(
        all_properties["correlation_dimension"][sort_inds],
        row,
        color=clr,
        linewidth=4
     )
plt.legend(results_reduced.index, facecolor='white', framealpha=1, frameon=True)
plt.title("Show legend")

# dg.better_savefig("../private_writing/fig_resources/lyap_legend.png", dpi=600)

